In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


from src.constants import TARGET_MAX_LENGHT, MAX_LENGHT_SOURCE
from src.data_utils.dataset import build_datset_train_val, VOCAB_SIZE, LHAND_IDX, LHAND_IDX, start_token_idx, end_token_idx, pre_process, pad_token_idx, FEATURE_COLUMNS
from src.prod_models.builder import build_prod_transformer_model_v2
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf
import numpy as np


TRIALS = 15
EPOCHS = 5000
EPOCHS_PER_TRIAL = 10
BATCH_SIZE = 128
TRAIN_SPLIT = 0.8
MODEL_NAME = "prod_v2"

ImportError: cannot import name 'FEATURES_SIZE' from 'src.constants' (/Users/matiasgonzalez/workspace/fingerspelling-recognition/src/constants.py)

In [ ]:
FEATURE_COLUMNS.shape[0]/2

In [ ]:
train_dataset, val_dataset = build_datset_train_val(split=TRAIN_SPLIT, batch_size=BATCH_SIZE)

In [ ]:
next(iter(train_dataset))

In [ ]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_prod_transformer_model_v2(trial=trial)
    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_PER_TRIAL, callbacks=get_predefine_callbacks(model_name=MODEL_NAME, patience=3), verbose=0)
    levenshtein = model.evaluate(val_dataset)[-1]

    return  levenshtein

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=TRIALS, gc_after_trial=True, show_progress_bar=True)

In [ ]:
trials = study.best_trials

In [ ]:
trials[0]

In [ ]:
tf.keras.backend.clear_session()
trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")

    model = build_prod_transformer_model_v2(trial=trial)

    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])

    print(model.summary())
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_predefine_callbacks(model_name=MODEL_NAME, patience=10))
   
    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{MODEL_NAME}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")

In [ ]:
from src.data_utils.dataset import char_to_num, num_to_char

In [ ]:
# target_sequence = [char_to_num[w] for w in ["<"]]

# for batch_index, batch in enumerate(val_dataset):
#     batch = batch[0]

#     sources = batch[0] #batch["source"]
#     targets = batch[1] #batch["target"]
    
#     print(sources.shape)
#     print(targets.shape)

#     for index_sample, (source, target) in enumerate(zip(sources, targets)):
#         source = tf.expand_dims(source, axis=0)
#         target_sequence = [char_to_num[w] for w in ["<"]]
#         y_true = "".join([num_to_char[w] for w in target.numpy()])
    
#         for i in range(TARGET_MAX_LENGHT):
#             next_token = tf.expand_dims(tf.pad(tf.constant(target_sequence),
#              [[0, TARGET_MAX_LENGHT-len(target_sequence)]],
#               mode='CONSTANT',
#                constant_values=pad_token_idx,
#                 name=None),
#                  axis=0)

#             print("next target sequence to predict: ", next_token)
#             y_pred = model((source, next_token))

#             y_pred = tf.cast(tf.argmax(y_pred, axis=2), dtype=tf.int32)

#             print("argmax:", y_pred)

#             mask = tf.not_equal(y_pred, pad_token_idx)
#             next_token = y_pred[mask][-1].numpy()

#             target_sequence.append(next_token)

#             print("sequence so far: ", "".join([num_to_char[w] for w in target_sequence]))
#             print("Label: ", y_true)

#             if num_to_char[next_token]==">":
#                 break

#         print(f"================================={index_sample}=========================================")
#         if index_sample==1:
#             break

#     if batch_index==1:
#         break

In [ ]:
model.summary()

In [ ]:
# Save model

model.save(f"../models/{MODEL_NAME}", save_format="tf")

# TF lite

In [ ]:
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, FEATURE_COLUMNS.shape[0]], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)

        x = x[None]

        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, FEATURE_COLUMNS.shape[0])), lambda: tf.identity(x))

        x = x[0]

        x = pre_process(x)
        #shape after [MAX_LENGHT_SOURCE, FEATURE_SIZE]

        x = x[None]

        x = self.model.generate(x)

        x = x[0]
        idx = tf.argmax(tf.cast(tf.equal(x, self.target_end_token_idx), tf.int32))
        idx = tf.where(tf.math.less(idx, 1), tf.constant(2, dtype=tf.int64), idx)
        x = x[1:idx]

        x = tf.one_hot(x, 59)
        return {"outputs": x}

tflitemodel_base = TFLiteModel(model)

In [ ]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = keras_model_converter.convert()

In [ ]:
import json

# with open('/kaggle/working/model.tflite', 'wb') as f:
with open("../models/model.tflite", "wb") as f:    
    f.write(tflite_model)

infargs = {"selected_columns" : list(FEATURE_COLUMNS)}

# with open("inference_args.json", "w") as json_file:
with open("../models/inference_args.json", "w") as json_file:
    json.dump(infargs, json_file)

In [ ]:
!zip submission.zip  '../models/model.tflite' '../models/inference_args.json'

# Test results

In [ ]:
# from batch 1

source_batch, target_batch = next(iter(val_dataset))[0]

In [ ]:
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

# interpreter = tf.lite.Interpreter("model.tflite")
interpreter = tf.lite.Interpreter("../models/model.tflite")

# with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)

rev_character_map = {j:i for i,j in character_map.items()}
found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner(REQUIRED_SIGNATURE)

prediction_str = ""
for source_element, target_element in zip(source_batch, target_batch):
    # print(tf.expand_dims(target_element, axis=0).numpy())

    output = prediction_fn(inputs=source_element)

    # print(output[REQUIRED_OUTPUT])

    # break

    print("generated: ", "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)]))
    print("target: ", "".join([rev_character_map.get(s, "") for s in target_element.numpy()]))

In [ ]:
interpreter.get_input_details()